In [ ]:
%pip install --upgrade databricks-sdk
dbutils.library.restartPython()

In [0]:
dbutils.widgets.removeAll()

In [0]:
dbutils.widgets.text("config", "/Workspace/Users/alex.lopes@databricks.com/WorkspaceInventory/conf/global_init_scripts.yml")
dbutils.widgets.text("catalog_schema", "shared.bigc")

# Account Inventory
dbutils.widgets.text("account_id", "00000000-0000-0000-0000-000000000000")
dbutils.widgets.text("azure_workspace_resource_id", "00000000-0000-0000-0000-000000000000")
dbutils.widgets.text("azure_tenant_id", "00000000-0000-0000-0000-000000000000")
dbutils.widgets.text("azure_client_id", "00000000-0000-0000-0000-000000000000")
dbutils.widgets.text("azure_client_secret", "00000000-0000-0000-0000-000000000000")


In [0]:
widgets = dbutils.widgets.getAll()

In [0]:
from databricks.sdk import WorkspaceClient, AccountClient
from pprint import pprint
import yaml
import traceback
import functools
from pyspark.sql.functions import *

catalog_schema = widgets["catalog_schema"]

In [0]:
print("Config:", widgets["config"])
with open(widgets["config"], "r") as f:
  config = yaml.safe_load(f.read())
pprint(config)

Config: /Workspace/Users/alex.lopes@databricks.com/WorkspaceInventory/conf/global_init_scripts.yml
[{'api': {'call': 'w.global_init_scripts.list',
          'write': {'mode': 'overwrite',
                    'table': 'shared.bigc.global_init_scripts'}}}]


In [0]:
CLOUD_AZURE = "Azure"
CLOUD_GCP = "GCP"
CLOUD_AWS = "AWS"
CLOUD_UNKNOWN = "Unknown"

def get_api_call(conf):
  api_func = conf['call'].split(".")
  print(api_func)

  objs = []
  keys = {}

  w = WorkspaceClient()
  workspace_url = w.config.host
  if "azuredatabricks.net" in workspace_url:
    cloud = CLOUD_AZURE
  elif "gcp.databricks.com" in workspace_url:
    cloud = CLOUD_GCP
  elif "databricks.com" in workspace_url:
    cloud = CLOUD_AWS
  else:
    cloud = CLOUD_UNKNOWN

  if api_func[0] == 'w':
    objs.append(w)
    keys["workspace_id"] = w.get_workspace_id()
    user = w.current_user.me().as_dict()
    keys["execution_user_name"] = user["displayName"]
    keys["execution_user_id"] = user["id"]
  elif api_func[0] == 'a':
    if cloud == CLOUD_AZURE :
      objs.append(AccountClient(
          host='https://accounts.azuredatabricks.net/',
          account_id=widgets["account_id"],
          azure_workspace_resource_id = widgets["azure_workspace_resource_id"],
          azure_tenant_id=widgets["azure_tenant_id"],
          azure_client_id=widgets["azure_client_id"],
          azure_client_secret=widgets["azure_client_secret"]
      ))
    elif cloud == CLOUD_GCP:
      objs.append(AccountClient())
    elif cloud == CLOUD_AWS:
      objs.append(AccountClient())


  for i, f in enumerate(api_func[1:]):
    objs.append(getattr(objs[i], f))
  return objs, keys


In [0]:
def create_df(df_result, rows, i, schema):
  if (i + 1) % 1000 == 0:
      # 1000: avoid the RecursionError: maximum recursion depth exceeded
      # https://docs.python.org/2/library/sys.html#sys.tracebacklimit
      print("Sending to DF data", i)
      tmp_df = spark.createDataFrame(rows, schema)
      if df_result is None:
        df_result = tmp_df
      else:
        df_result = df_result.union(tmp_df)
      rows = []
  return df_result, rows

def execute_and_save(conf, objs, kargs={}):
  df_result = None
  rows = []
  schema = None
  i = 0
  if len(kargs) > 0:
    items = objs[-1](*conf.get("cargs", []), **kargs, **conf.get("kargs", {}))
  else:
    items = objs[-1](*conf.get("cargs", []), **conf.get("kargs", {}))

  # Handle non-iterable case
  if not hasattr(items, '__iter__'):
    items = [items]

  for item in items:
    try:
      row = item.as_dict()
      # Handle access control list case
      assert 'access_control_list' in row

      schema_keys = set()
      for acl in row['access_control_list']:
        object_id = row['object_id']
        object_type = row.get('object_type', '')

        # Dynamically add additional attributes
        fixed_col = {}
        for key, value in acl.items():
          if key not in ['all_permissions']:
            fixed_col[key] = value
        # Handle all_permissions
        assert 'all_permissions' in acl
        for permission in acl['all_permissions']:
          acl_row = {
            'object_id': object_id,
            'object_type': object_type,
            'permission': permission,
          }
          acl_row.update(fixed_col)
          # get all keys in acl_row
          schema_keys.update(acl_row.keys())
        rows.append(acl_row)
        i += 1

      schema = ",".join([i + " string" for i in schema_keys])
      df_result, rows = create_df(df_result, rows, i, schema)

    except Exception as e:
      print("Error in one object:", e, traceback.format_exc())
      continue

  print("Total items found:", i)
  if i > 0:
    tmp_df = spark.createDataFrame(rows, schema)
    df_result = df_result.union(tmp_df) if df_result is not None else tmp_df

  return df_result


In [0]:
def write_to_table(conf, df, keys, write_mode=None):
  df = df.withColumn("api_updated_at", current_timestamp())
  for key, value in keys.items():
    df = df.withColumn("api_" + key, lit(value))
  df = df.write
  write_conf = conf['write']
  if write_mode is None and "mode" in write_conf:
    df = df.mode(write_conf['mode'])
  elif write_mode:
    df = df.mode(write_mode)
  if "options" in write_conf:
    df = df.options(**write_conf['options'])


  df.saveAsTable(widgets["catalog_schema"] + '.' + write_conf['table'])

In [0]:
from itertools import product

for conf in config:
  conf = conf['api']
  print("Executing:", conf["call"], "Table:", conf["write"]["table"])
  objs, keys = get_api_call(conf)
  depend_kargs = {}
  if 'depend_kargs' in conf:
    param_names = conf["depend_kargs"].keys()
    target_table_columns = set()
    target_tables = set()
    for k, v in conf["depend_kargs"].items():
      # k is the params for execute_and_save, v is the table_name.column_name's values
      # create a dict of kargs for execute_and_save
      table_name, column_name = v.split(".")

      df = spark.table(f"{catalog_schema}.{table_name}")
      df = df.filter(df[column_name].isNotNull())

      # filter the df by filter_kargs
      if 'filter_kargs' in conf:
        for key, value in conf["filter_kargs"].items():
          df = df.filter(df[key] == value)

      df = df.select(column_name).distinct().collect()
      depend_kargs[k] = [row[0] for row in df] # {param_name1: [values], param_name2: [values], ...}
  # transform {param_name1: [v1, v5], param_name2: [v2, v3, v4], ...} to a combination of all values
  # {param_name1: [v1, v5], param_name2: [v2, v3, v4], ...} -> [[v1, v2], [v1, v3], [v1, v4], [v5, v2], [v5, v3], [v5, v4]]
  # use itertools.product to generate all combinations

  depend_kargs_combinations = list(product(*depend_kargs.values()))

  buffer_dfs = []
  batch_limit = 100
  batch_count = 0
  batch_index = 0

  final_df = None
  write_mode = None
  for combination in depend_kargs_combinations:
    print(f"Processing combination: {combination}")
    try:
      df = execute_and_save(conf, objs, dict(zip(depend_kargs.keys(), combination)))
      # if final_df is None:
      #   final_df = df
      # else:
      #   final_df = final_df.unionByName(df, allowMissingColumns=True)
      if df is not None:
        buffer_dfs.append(df)
        batch_count += 1
    except Exception as e:
      print("Error in combination: ", combination, e)
      continue

    if batch_count >= batch_limit:
      # avoid the RecursionError: maximum recursion depth exceeded
      # https://docs.python.org/2/library/sys.html#sys.tracebacklimit
      print(f"Writing batch {batch_index} with {batch_count} partitions")

      if len(buffer_dfs) == 1:
          merged_df = buffer_dfs[0]
      elif len(buffer_dfs) > 1:
          merged_df = functools.reduce(lambda a, b: a.unionByName(b, allowMissingColumns=True), buffer_dfs)
      else:
          print("No DataFrames to merge")
          merged_df = None


      if batch_index != 0:
          # if batch_index is 0, write_mode will be the config's mode
          # if batch_index is not 0, write_mode will be 'append' to write the data to the table batch by batch
          write_mode = 'append'
      write_to_table(conf, merged_df, keys, write_mode=write_mode)

      buffer_dfs = []
      batch_count = 0
      batch_index += 1

  if buffer_dfs:
    print(f"Writing final batch with {batch_count} partitions")

    if len(buffer_dfs) == 1:
        merged_df = buffer_dfs[0]
    elif len(buffer_dfs) > 1:
        merged_df = functools.reduce(lambda a, b: a.unionByName(b, allowMissingColumns=True), buffer_dfs)
    else:
        print("No DataFrames to merge")
        merged_df = None

    write_to_table(conf, merged_df, keys, write_mode=write_mode)

Executing: w.global_init_scripts.list Table: shared.bigc.global_init_scripts
['w', 'global_init_scripts', 'list']
Total items found: 0
No data to save
